# architecture-10 ( Basic Regression Wide And Deep Add TA-Lib )
What's new:

1- Add TA-Lib to wide section

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Concatenate, GlobalAveragePooling1D, LayerNormalization
import talib

In [20]:
# === Load and preprocess ===
df = pd.read_csv('datasets-10/XAGUSD-H1-rates.csv', sep='\t')

In [21]:
def add_ta_features(df):
    df['rsi'] = talib.RSI(df['<CLOSE>'], timeperiod=14)
    df['macd'], _, df['MACD_hist'] = talib.MACD(df['<CLOSE>'])
    df['ema_50'] = talib.EMA(df['<CLOSE>'], timeperiod=50)
    df['atr'] = talib.ATR(df['<HIGH>'], df['<LOW>'], df['<CLOSE>'], timeperiod=14)
    df['adx'] = talib.ADX(df['<HIGH>'], df['<LOW>'], df['<CLOSE>'], timeperiod=14)
    return df.dropna()


df = add_ta_features(df)

In [22]:
# Scale sequence features
sequence_features = ['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']
wide_features = ['<SPREAD>', '<VOL>', 'adx', 'atr', 'ema_50', 'macd', 'MACD_hist', 'rsi']

scaler_seq = MinMaxScaler()
scaler_wide = MinMaxScaler()
scaler_close = MinMaxScaler()

scaled_seq = scaler_seq.fit_transform(df[sequence_features])
scaled_wide = scaler_wide.fit_transform(df[wide_features])
scaled_close = scaler_close.fit_transform(df[['<CLOSE>']])

# === Create sequences ===
def create_sequences(seq_data, wide_data, target_data, window, horizon):
    X_seq, X_wide, y = [], [], []
    for i in range(window, len(seq_data) - horizon):
        X_seq.append(seq_data[i-window:i])
        X_wide.append(wide_data[i-1])  # use last row of each window
        y.append(target_data[i:i+horizon].flatten())
    return np.array(X_seq), np.array(X_wide), np.array(y)

WINDOW_SIZE = 60
FORECAST_HORIZON = 10

X_seq, X_wide, y = create_sequences(scaled_seq, scaled_wide, scaled_close, WINDOW_SIZE, FORECAST_HORIZON)

# === Train/Test split ===
X_seq_train, X_seq_test, X_wide_train, X_wide_test, y_train, y_test = train_test_split(
    X_seq, X_wide, y, test_size=0.2, shuffle=False
)

# === Build Wide & Deep model ===
def build_wide_deep_model(seq_shape, wide_shape, horizon):
    # Deep part
    seq_input = Input(shape=seq_shape, name="sequence_input")
    x = LSTM(64, return_sequences=True)(seq_input)
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)

    # Wide part
    wide_input = Input(shape=wide_shape, name="wide_input")

    # Combine
    combined = Concatenate()([x, wide_input])
    x = Dense(64, activation='relu')(combined)
    x = Dense(32, activation='relu')(x)
    output = Dense(horizon)(x)

    model = Model(inputs=[seq_input, wide_input], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_wide_deep_model(
    seq_shape=(X_seq_train.shape[1], X_seq_train.shape[2]),
    wide_shape=(X_wide_train.shape[1],),
    horizon=FORECAST_HORIZON
)


In [23]:
import csv
# Choose output path
summary_path = 'saved_models/model_summary.csv'

# Open CSV for writing
with open(summary_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Header
    writer.writerow(['Layer Name', 'Layer Type', 'Output Shape', 'Param #'])

    # Iterate over model layers
    for layer in model.layers:
        layer_name = layer.name
        layer_type = layer.__class__.__name__
        output_shape = layer.output_shape if hasattr(layer, 'output_shape') else 'N/A'
        param_count = layer.count_params()

        writer.writerow([layer_name, layer_type, output_shape, param_count])

print(f'Model summary saved to: {summary_path}')


Model summary saved to: saved_models/model_summary.csv


In [ ]:
# === Train ===
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    [X_seq_train, X_wide_train],
    y_train,
    validation_split=0.2,
    epochs=50,
    callbacks=[es],
    verbose=1
)

In [ ]:
# === Load the last 110 rows from a separate CSV file for prediction ===
input_df = pd.read_csv('datasets-10/new-data-for-test/rows-60-from-20240503/rows-110-from-20240503.csv', sep='\t')

input_df = add_ta_features(input_df)

# Take only the last 60 usable rows after TA features
input_df = input_df.tail(60)

input_seq_scaled = scaler_seq.transform(input_df[sequence_features])
input_wide_scaled = scaler_wide.transform(input_df[wide_features])

input_seq = np.expand_dims(input_seq_scaled, axis=0)
input_wide = np.expand_dims(input_wide_scaled[-1], axis=0)

pred = model.predict([input_seq, input_wide])
prediction = scaler_close.inverse_transform(pred)
print("Next 10 predicted closing prices:", prediction.flatten())

# plot section

In [ ]:
import sys

sys.path.insert(1, '../utils')
import forex_plot_utils
import os

# PARAMETERS
csv1_path = 'datasets-10/new-data-for-test/rows-60-from-20240503/latest-4-for-history.csv'
csv3_path = 'datasets-10/new-data-for-test/rows-60-from-20240503/after.csv'
plot_title = 'Actual vs Predicted Forex Closing Prices'
output_plot_path = None  # e.g., 'output.png'

# LOAD DATA FROM CSVS
historical_df = forex_plot_utils.load_csv_with_datetime(csv1_path) if os.path.exists(csv1_path) else None
actual_future_df = forex_plot_utils.load_csv_with_datetime(csv3_path) if os.path.exists(csv3_path) else None

# LOAD DATA FROM PREDICTION

# Combine <DATE> and <TIME> columns into a datetime
input_df['DATETIME'] = pd.to_datetime(input_df['<DATE>'] + ' ' + input_df['<TIME>'])

last_timestamp = input_df['DATETIME'].iloc[-1]
datetime_index = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), periods=len(prediction[0]), freq='h')

# Create DataFrame
predicted_df = pd.DataFrame({'DATETIME': datetime_index, '<CLOSE>': prediction[0]})

In [ ]:
# PLOT
forex_plot_utils.plot_all_series(
    historical_df=historical_df,
    predicted_df=predicted_df,
    actual_future_df=actual_future_df,
    title=plot_title,
    output_path=output_plot_path
)

In [ ]:
from datetime import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt

# === Create timestamp and paths ===
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f'model_{timestamp}.keras'
model_path = os.path.join('saved_models', model_filename)

# Directory to hold logs and extras
log_dir = os.path.join('saved_models', f'model_{timestamp}_logs')
os.makedirs(log_dir, exist_ok=True)

# === Save model ===
model.save(model_path)

# === Save training history ===
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(log_dir, 'training_history.csv'), index=False)

# === Save training loss plot ===
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(log_dir, 'training_loss.png'))
plt.close()

# === Save model summary and final performance ===
with open(os.path.join(log_dir, 'model_log.txt'), 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    final_train_loss = history.history['loss'][-1]
    final_test_loss, final_test_mae = model.evaluate([X_seq_test, X_wide_test], y_test, verbose=0)
    f.write(f'\nFinal Training Loss: {final_train_loss:.6f}\n')
    f.write(f'Final Test Loss : {final_test_loss:.6f}\n')
    f.write(f'Final Test MAE : {final_test_mae:.6f}\n')


